In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn_tda as sktda
import statmapper as stm
import networkx as nx

from sklearn.metrics import pairwise_distances
from mpl_toolkits.mplot3d import Axes3D
from stochmapper import *

# Annulus

## Dataset

### Underlying manifold

In [ ]:
num_pts = 5000
radius  = 1.

In [ ]:
theta   = np.random.uniform(low=0., high=2*np.pi, size=num_pts)
xs, ys  = radius * np.cos(theta), radius * np.sin(theta)
noise_x = np.random.normal(loc=xs, scale=.1, size=num_pts)
noise_y = np.random.normal(loc=ys, scale=.1, size=num_pts)
X       = np.hstack([np.reshape(xs+noise_x, [-1,1]), np.reshape(ys+noise_y, [-1,1])])

In [ ]:
%matplotlib notebook
plt.scatter(X[:,0], X[:,1], s=3)
plt.show()

In [ ]:
delta = sktda.estimate_scale(X, 100)
print(delta)

### Probability distributions

#### Ideal case

In [ ]:
distributions = []

In [ ]:
num_samples = 1000

Gaussian distribution.

In [ ]:
for i in range(num_pts):
    distributions.append(np.random.normal(loc=X[i,0], scale=10, size=num_samples))

Bimodal distribution.

In [ ]:
num_pts=0
for i in range(num_pts,num_pts+1):
    distrib = []
    for s in range(num_samples):
        idx = np.random.choice(2, 1)
        if idx == 0:
            distrib.append(np.random.normal(loc=X[i,0], scale=20, size=1))
        else:
            distrib.append(np.random.normal(loc=-X[i,0], scale=20, size=1))
    distributions.append(distrib)

#### Filter values of nearest neighbors

In [ ]:
real = []
for i in range(num_pts):
    real.append(np.random.normal(loc=X[i,0], scale=10, size=1)[0])

In [ ]:
real = []
for i in range(num_pts):
    real.append(np.random.normal(loc=0, scale=X[i,0]-np.min(X[i,:]), size=1)[0])

In [ ]:
real = []
for i in range(num_pts):
    idx = np.random.choice(2, 1)
    if idx == 0:
        real.append(np.random.normal(loc=X[i,0], scale=10, size=1)[0])
    else:
        real.append(np.random.normal(loc=-X[i,0], scale=10, size=1)[0])

In [ ]:
distributions = infer_distributions_from_neighborhood(real, X, 3*delta, "point cloud")

#### Distances between distributions

In [ ]:
m, M = np.min([np.min(d) for d in distributions]), np.max([np.max(d) for d in distributions])
print(m,M)

In [ ]:
eud = EuclideanDistance(num_bins=100, bnds=(m, M))
dists = eud.compute_matrix(distributions)

In [ ]:
erw = EntropyRegularizedWasserstein(epsilon=1e-3, num_bins=100, bnds=(-10, 10))
dists = erw.compute_matrix(distributions)

In [ ]:
w1d = Wasserstein1D(p=1, num_bins=100, bnds=(m, M))
dists = w1d.compute_matrix(distributions)

In [ ]:
kld = KullbackLeiblerDivergence(num_bins=1000, bnds=(m, M))
dists = kld.compute_matrix(distributions)

#### Estimate Lipschitz coefficient

In [ ]:
D = euclidean_distances(X)
coeff = 0
for i in range(num_pts):
    for j in range(i+1, num_pts):
        if D[i,j] <= delta and D[i,j] > 0:
            coeff = max(coeff, dists[i,j]/D[i,j])
print(coeff)

### Visualization

In [ ]:
plt.figure()
plt.hist(np.array(distributions[0]), bins=100)
plt.show()

Visualize a given realization.

In [ ]:
z = [distributions[i][0] for i in range(num_pts)]

In [ ]:
z = [np.mean(distrib) for distrib in distributions]

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax  = fig.add_subplot(111, projection="3d")
ax.scatter(X[:,0], X[:,1], z, s=1.)
plt.show()

## Single realization Mapper

In [ ]:
mapper = sktda.MapperComplex(
    filters=np.reshape(np.array(real), [-1,1]), 
    filter_bnds=np.array([[np.nan, np.nan]]),
    resolutions=np.array([5]), gains=np.array([.3]), colors=X[:,0:1],
    clustering=AgglomerativeClustering(n_clusters=None, linkage="single", distance_threshold=1.)
                            ).fit(X)

In [ ]:
G = stm.mapper2networkx(mapper)
nx.draw_networkx(G, with_labels=False,
                 node_color=[mapper.node_info_[name]["colors"][0] for name in G.nodes()])
                 #node_size=[len(mapper.node_info_[name]["indices"]) for name in G.nodes()])

## Mean stochastic Mapper 

In [ ]:
mapper = MeanStochasticMapperComplex(
    filters=distributions, infer_distributions=False, threshold=1.,
    resolution=7, gain=.3, colors=X[:,0:1],
    clustering=AgglomerativeClustering(n_clusters=None, linkage="single", distance_threshold=delta)
                            ).fit(X)

In [ ]:
G = stm.mapper2networkx(mapper)
nx.draw_networkx(G, with_labels=False,
                 node_color=[mapper.node_info_[name]["colors"][0] for name in G.nodes()])
                 #node_size=[len(mapper.node_info_[name]["indices"]) for name in G.nodes()])

## Exact stochastic Mapper

In [ ]:
print(np.median(dists))

In [ ]:
mapper = StochasticMapperComplex(
    filters=distributions,
    codomain="distributions", infer_distributions=False, #threshold=1.,
    #cover=VoronoiCover(n_patches=10, threshold=delta/3), 
    #distance=EuclideanDistance(num_bins=100, bnds=(m,M)), #distance=Wasserstein1D(num_bins=100, bnds=(m,M), p=1),
    cover=kPDTMCover(n_patches=10, h=3, threshold=delta/10, tol=1e-8, num_bins=100, bnds=(m,M)),
    #cover=EuclideanKMeansCover(n_patches=10, threshold=delta/10, num_bins=100, bnds=(m,M)),
    #cover=WassersteinKMeansCover(n_patches=10, threshold=.01, num_bins=100, bnds=(m,M)),
    colors=np.reshape(X[:,0], [-1,1]),
    clustering=AgglomerativeClustering(n_clusters=None, linkage="single", distance_threshold=delta)
                            ).fit(X)

In [ ]:
G = stm.mapper2networkx(mapper)
nx.draw_networkx(G, with_labels=False,
                 node_color=[mapper.node_info_[name]["colors"][0] for name in G.nodes()])
                 #node_size=[len(mapper.node_info_[name]["indices"]) for name in G.nodes()])